In [11]:
# TVBS
# 爬蟲套件
import requests
from bs4 import BeautifulSoup
import json
from fake_useragent import UserAgent
# 資料整理類套件
import pandas as pd
import re
import time
import random
import datetime
from tomorrow import threads
from pymongo import MongoClient
from datetime import datetime
import pymongo

In [12]:
ua = UserAgent()
headers = {"User-Agent":ua.random}

In [13]:
def GetAllLinks():
    """第一層所有links"""
    links = []
    page = 1
    while True:
        url = "https://news.tvbs.com.tw/news/searchresult/news/"+str(page)+"/?search_text=政治"
        res = requests.get(url,headers=headers)
        soup = BeautifulSoup(res.text)
        if page > 30:
            break
        try:
            li = soup.find("div",class_="search_list_div").find_all("a")
        except AttributeError:
            break
        for i in li:
            links.append(i.get("href"))
        page += 1

    return links

In [14]:
def GetTVBS(response):
    """爬內文"""
    soup = BeautifulSoup(response.text)
    Title = soup.find("h1",class_="margin_b20").text.replace("\u3000"," ")
    CreateTime = soup.find("div",class_="icon_time time leftBox2").text
    CreateTime = datetime.strptime(CreateTime,'%Y/%m/%d %H:%M')
    Content = soup.find("div",class_="h7 margin_b20").find("p").text
    Tag = []
    for i in soup.find("div",class_="adWords").find_all("a"):
        Tag.append(i.text)
    try:
        Author = soup.find("h4",class_="font_color5 leftBox1").find("a").text
    except:
        Author = "無作者"
    Url = soup.find("meta",itemprop="url").get("content")
    ndf = pd.DataFrame(data = [{"Source":"News",
                                "Site":"TVBS",
                                "Url":Url,
                                "Title":Title,
                                "Author":Author,
                                "CreateTime":CreateTime,
                                "Content":Content,
                                "Tag":Tag}],
                      columns = ['Source', 'Site', 'Url', 'CreateTime', 'Author', 'Title', 'Content','Tag'])
    print(Url)
    return ndf

In [16]:
# testurl = 'https://news.tvbs.com.tw/world/1238037'
# response = requests.get(testurl,headers=headers)
# GetTVBS(response)

In [17]:
@threads(100)
def MultiThread_Crawl(url):
    try:
        return requests.get(url, headers=headers)
    except:
        pass

In [18]:
def CrawlAllNews(links):
    # 爬取全部連結的新聞內容
    print('There are {} links in pages.'.format(len(links)))
    
    # 多線程爬蟲
    responses = [MultiThread_Crawl(link) for link in links]

    # 整理成DataFrame
    allData = []
    for response in responses:
        try:
            ndf = GetTVBS(response)
            allData.append(ndf)
        except:
            pass
    df = pd.concat(allData, ignore_index=True)
    print('There are {} News in DataFrame.'.format(len(df)))
    return df

In [19]:
def dataframe_to_mongo_test(select_df):
    user_name = "test"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "test"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.test   #DB_name
    collection = db.test_news  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")

    

In [20]:
def dataframe_to_mongo(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "192.168.1.126"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.news_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")
        

In [21]:
def dataframe_to_mongo_political(select_df):
    user_name = "pol"
    password = "pn123456"
    connect_IP = "ntcpn108.ddns.net"
    DB_name = "political"
    mode = "SCRAM-SHA-1"
    
    url = "mongodb://"+str(user_name)+":"+str(password)+"@"+str(connect_IP)+":27017/?authSource="+str(DB_name)+"&authMechanism="+str(mode)
    client = MongoClient(url)
    db = client.political   #DB_name
    collection = db.news_pol  #collection_name
    
    try:
        print("total "+str(len(select_df))+" into database")
        records = select_df.to_dict('records')
        collection.insert_many(records,ordered=False)

    except pymongo.errors.BulkWriteError as e:
        print("already "+str(len(e.details['writeErrors']))+" duplicated wont into database")
        print(str(len(select_df) - len(e.details['writeErrors']))+" into database")
     

In [25]:
links = GetAllLinks() 
df = CrawlAllNews(links)
dataframe_to_mongo_political(df)